# 1. Описание задачи

В рамках предмета «Анализ неструктурированных данных» с целью получения практических навыков работы с текстами и API был выполнен проект по составлению собственного корпуса технических новостей с сайта HackerNews (https://news.ycombinator.com/). Это веб-агрегатор, где пользователь может опубликовать новостной пост, задать интересующий вопрос (ASK HN), поделиться чем-то с комьюнити (Show HN) или откликнуться на какую-либо из предложенных в отдельном разделе сайта вакансий.

Согласно техническому заданию, необходимо было получить посты за последние полгода вместе с информацией о них (комментарии, число лайков, дата публикации и ссылки на внешние ресурсы), пройти по всем найденным в полученных постах ссылкам с последующим сохранением страниц в виде текстовых сообщений, выделить в них упоминающие определенный бренд и сделать краткий анализ извлеченных данных.

In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import re
import time
from bs4 import BeautifulSoup
from hackernews import HackerNews
from nltk import FreqDist
from nltk.corpus import stopwords
from readability import Document
from urllib.parse import urlparse

stop_words = stopwords.words('english')

# 2. Сбор последних постов

Для работы с HackerNews использовался найденный враппер для официального API рассматриваемого ресурса (https://github.com/HackerNews/API). Он доступен по следующей ссылке: https://github.com/avinassh/haxor.

С его помощью были извлечены новейшие посты, их id, названия, даты публикации, содержащиеся в них внешние ссылки, комментарии и число лайков.

In [95]:
hn = HackerNews()

Из описания официального API HackerNews:

> Up to 500 top and new stories are at /v0/topstories and /v0/newstories.

Таким образом, при сборе данных с помощью API нельзя извлечь более 500 последних публикаций. Та же история с используемым враппером. Попытаемся достать больше 500 постов --- скажем, 1000. 

In [98]:
new = hn.new_stories(1000)

In [99]:
len(new)

500

Видно, что по запросу 1000 постов выдалось всего 500. Значит, к сожалению, вместо данных за последние полгода придется довольствоваться публикациями чуть больше, чем за последние сутки.

In [152]:
# creating dataframe for the parsed data
columns = ['id', 'title', 'date', 'url', 'comments', 'likes']
df = pd.DataFrame(columns=columns)

In [153]:
start_time = time.time()
for s in new:
    # getting item by id
    item = hn.get_item(s)
    print('\rProcessing item {}, {} of {} ({}% completed)'.format(
                s, new.index(s) + 1, len(new),
                round((new.index(s) + 1) / len(new) * 100, 2)), end='')
    # all comments are going to be stored in the list
    comments = []
    # checking for comments
    if item.descendants > 0:
        # collecting comments
        for k in item.kids:
            c = hn.get_item(k).text
            comments.append(c)
    # adding new row to the existing dataframe with all gathered information
    df.loc[len(df)] = [str(s), item.title, item.submission_time, item.url, comments, int(item.score)]
print('\nTime spent: {} minutes'.format(round((time.time() - start_time) / 60, 2)))

Processing item 15422989, 500 of 500 (100.0% completed)
Time spent: 3.99 minutes


In [157]:
df['likes'] = df['likes'].astype(int)

In [160]:
df.head()

,id,title,date,url,comments,likes
0,15427629,"Type Safety, ORM and Dependency Injection with...",2017-10-08 12:55:27,http://paulosuzart.github.io/blog/2017/10/04/t...,[The syntax for the ORM definition and the rel...,2
1,15427621,Get Ahead in Tech… by Reading on Saturdays,2017-10-08 12:52:46,https://hackernoon.com/get-ahead-in-tech-by-re...,[],1
2,15427607,Comparing the Performance Between Native iOS (...,2017-10-08 12:49:15,https://www.codementor.io/jcalderaio/comparing...,[],1
3,15427604,"Origins and History of Unix, 1969-1995",2017-10-08 12:47:38,http://www.catb.org/esr/writings/taoup/html/ch...,[],1
4,15427601,New Earbuds by Google Translates 40 Languages ...,2017-10-08 12:46:09,http://guardian.ng/life/whatsnew/new-earbuds-b...,[Things are getting closer to black mirror.],4


In [161]:
df.tail()

,id,title,date,url,comments,likes
495,15423032,We should talk about gun control vs. I’m going...,2017-10-07 12:21:47,https://www.duffelblog.com/2017/10/gun-control...,[],2
496,15423027,"Physics, Topology, Logic and Computation: A Ro...",2017-10-07 12:19:48,https://arxiv.org/abs/0903.0340,[John Baez is a fantastic writer. His home pag...,129
497,15423017,Angular News – Sharing Top Content from the An...,2017-10-07 12:15:18,https://angular.jsnews.io/?utm_source=social&u...,[],1
498,15422995,Show HN: Generate avatars from user initials,2017-10-07 12:05:12,https://ui-avatars.com,[Two of my favorite avatar generators:<p><a hr...,158
499,15422989,Ask HN: If the world were flat would it have o...,2017-10-07 12:03:55,None,[],1


# 3. Обход внешних ссылок

После сбора данных был написан краулер, переходящий по всем найденным внешним ссылкам. Для получения содержимого страницы использовался модуль requests (https://github.com/requests/requests). В процессе обхода каждая рассматриваемая страница сохранялась в виде текстового сообщения с помощью инструмента readability (https://github.com/buriy/python-readability).

In [195]:
# crawler function
def url2text(url):
    # checking for url presence in the news post
    if url:
        # try-except construction here because some websites are restricted and have limited access
        try:
            # gaining url content
            response = requests.get(url)
            # building a etree document out of html
            doc = Document(response.text)
            # returning cleaned up content
            return doc.summary()
        except:
            return '.'
    else:
        return ''

In [197]:
start_time = time.time()
df['url_summary'] = df['url'].apply(lambda u: url2text(u))
print('\nTime spent: {} minutes'.format(round((time.time() - start_time) / 60, 2)))

c:\programdata\anaconda3\lib\site-packages\requests\packages\urllib3\connection.py:340: SubjectAltNameWarning: Certificate for images.guide has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SubjectAltNameWarning



Time spent: 12.41 minutes


In [199]:
df.head()

,id,title,date,url,comments,likes,url_summary
0,15427629,"Type Safety, ORM and Dependency Injection with...",2017-10-08 12:55:27,http://paulosuzart.github.io/blog/2017/10/04/t...,[The syntax for the ORM definition and the rel...,2,"<html><body><div><article class=""post-content""..."
1,15427621,Get Ahead in Tech… by Reading on Saturdays,2017-10-08 12:52:46,https://hackernoon.com/get-ahead-in-tech-by-re...,[],1,"<html><body><div><div class=""section-inner sec..."
2,15427607,Comparing the Performance Between Native iOS (...,2017-10-08 12:49:15,https://www.codementor.io/jcalderaio/comparing...,[],1,"<html><body><div><div class=""article__content""..."
3,15427604,"Origins and History of Unix, 1969-1995",2017-10-08 12:47:38,http://www.catb.org/esr/writings/taoup/html/ch...,[],1,"<html><body><div><div class=""sect1"" lang=""en"">..."
4,15427601,New Earbuds by Google Translates 40 Languages ...,2017-10-08 12:46:09,http://guardian.ng/life/whatsnew/new-earbuds-b...,[Things are getting closer to black mirror.],4,"<html><body><div><article data-post-type=""post..."


In [200]:
df.tail()

,id,title,date,url,comments,likes,url_summary
495,15423032,We should talk about gun control vs. I’m going...,2017-10-07 12:21:47,https://www.duffelblog.com/2017/10/gun-control...,[],2,"<body id=""readabilityBody"">\n<center><h1>403 F..."
496,15423027,"Physics, Topology, Logic and Computation: A Ro...",2017-10-07 12:19:48,https://arxiv.org/abs/0903.0340,[John Baez is a fantastic writer. His home pag...,129,"<html><body><div><div class=""dateline""><p>(Sub..."
497,15423017,Angular News – Sharing Top Content from the An...,2017-10-07 12:15:18,https://angular.jsnews.io/?utm_source=social&u...,[],1,"<html><body><div><div class=""listing listing-b..."
498,15422995,Show HN: Generate avatars from user initials,2017-10-07 12:05:12,https://ui-avatars.com,[Two of my favorite avatar generators:<p><a hr...,158,"<html><body><div><div class=""container"">\n\t\t..."
499,15422989,Ask HN: If the world were flat would it have o...,2017-10-07 12:03:55,None,[],1,


В процессе сбора данных некоторые ссылки не были обработаны, как, например, указанная ниже — в силу того, что социальная сеть LinkedIn на данный момент заблокирована на территории РФ.

In [196]:
url2text('https://www.linkedin.com/pulse/how-we-built-managed-full-mesh-transit-vpc-vpn-aws-sascha-coldewey/')

'.'

Подобных случаев было 7. Также в 30 постах не было найдено внешних ссылок, ибо, как видно из нижней строки полученной таблицы, в выборку попали посты типа Ask HN, содержащие в себе не новости, а вопросы, которые пользователи задавали комьюнити. Ссылки там на что-либо, соответственно, отсутствовали.

In [202]:
df['url_summary'].value_counts()[''], df['url_summary'].value_counts()['.']

(30, 7)

# 4. Фильтрация сообщений по бренду

После получения сообщений из них были выделены те, в которых упоминается Youtube. Данный бренд был выбран ввиду своей популярности на основе данных за прошлый год от компании Infegy, занимающейся мониторингом и анализом различных социальных медиа (https://top50.infegy.com/).

In [210]:
start_time = time.time()
df['youtube'] = df['url_summary'].apply(lambda u: 'youtube' in u.lower())
print('Time spent: {} minutes'.format(round((time.time() - start_time) / 60, 2)))

Time spent: 0.0 minutes


In [211]:
df.head()

,id,title,date,url,comments,likes,url_summary,youtube
0,15427629,"Type Safety, ORM and Dependency Injection with...",2017-10-08 12:55:27,http://paulosuzart.github.io/blog/2017/10/04/t...,[The syntax for the ORM definition and the rel...,2,"<html><body><div><article class=""post-content""...",False
1,15427621,Get Ahead in Tech… by Reading on Saturdays,2017-10-08 12:52:46,https://hackernoon.com/get-ahead-in-tech-by-re...,[],1,"<html><body><div><div class=""section-inner sec...",False
2,15427607,Comparing the Performance Between Native iOS (...,2017-10-08 12:49:15,https://www.codementor.io/jcalderaio/comparing...,[],1,"<html><body><div><div class=""article__content""...",False
3,15427604,"Origins and History of Unix, 1969-1995",2017-10-08 12:47:38,http://www.catb.org/esr/writings/taoup/html/ch...,[],1,"<html><body><div><div class=""sect1"" lang=""en"">...",False
4,15427601,New Earbuds by Google Translates 40 Languages ...,2017-10-08 12:46:09,http://guardian.ng/life/whatsnew/new-earbuds-b...,[Things are getting closer to black mirror.],4,"<html><body><div><article data-post-type=""post...",True


In [212]:
df.tail()

,id,title,date,url,comments,likes,url_summary,youtube
495,15423032,We should talk about gun control vs. I’m going...,2017-10-07 12:21:47,https://www.duffelblog.com/2017/10/gun-control...,[],2,"<body id=""readabilityBody"">\n<center><h1>403 F...",False
496,15423027,"Physics, Topology, Logic and Computation: A Ro...",2017-10-07 12:19:48,https://arxiv.org/abs/0903.0340,[John Baez is a fantastic writer. His home pag...,129,"<html><body><div><div class=""dateline""><p>(Sub...",False
497,15423017,Angular News – Sharing Top Content from the An...,2017-10-07 12:15:18,https://angular.jsnews.io/?utm_source=social&u...,[],1,"<html><body><div><div class=""listing listing-b...",True
498,15422995,Show HN: Generate avatars from user initials,2017-10-07 12:05:12,https://ui-avatars.com,[Two of my favorite avatar generators:<p><a hr...,158,"<html><body><div><div class=""container"">\n\t\t...",False
499,15422989,Ask HN: If the world were flat would it have o...,2017-10-07 12:03:55,None,[],1,,False


In [213]:
df['youtube'].value_counts()

False    454
True      46
Name: youtube, dtype: int64

46 из 500 последних постов содержат внешние ссылки на страницы, в тексте которых упоминается Youtube.

In [262]:
df.to_csv('hn_newest.csv')

# 5. Краткий анализ полученного корпуса

## 5.1. Количество постов

In [222]:
# selecting posts with internal links containing 'youtube'
data = df[df['youtube'] == True].drop('youtube', axis=1)

In [223]:
len(data)

46

Итак, в собранном корпусе 46 постов.

In [224]:
data

,id,title,date,url,comments,likes,url_summary
4,15427601,New Earbuds by Google Translates 40 Languages ...,2017-10-08 12:46:09,http://guardian.ng/life/whatsnew/new-earbuds-b...,[Things are getting closer to black mirror.],4,"<html><body><div><article data-post-type=""post..."
7,15427592,Believer's Voice of Victory Network Live Strea...,2017-10-08 12:42:57,http://jjjvirtual7.blogspot.com/2017/10/believ...,[],1,"<html><body><div><div class=""post-body entry-c..."
8,15427584,Inside the internet rehab,2017-10-08 12:38:55,https://www.theguardian.com/technology/2017/ju...,[],1,"<html><body><div><div class=""content__article-..."
9,15427581,New digital music platform for Afrika,2017-10-08 12:35:59,https://www.iafrikan.com/2017/10/08/okayafrica...,[],1,"<html><body><div><div class=""post-content inne..."
14,15427526,How to fail in a coding interview which you ca...,2017-10-08 12:08:57,https://medium.com/@l1feh4ck/how-to-fail-in-a-...,[],2,"<html><body><div><div class=""section-inner sec..."
16,15427497,Advice for New and Junior Data Scientists,2017-10-08 11:54:52,https://medium.com/@rchang/advice-for-new-and-...,[],1,"<html><body><div><div class=""section-inner sec..."
45,15427322,Should Cognitect Do More for Clojure?,2017-10-08 10:25:44,http://www.lispcast.com/cognitect-clojure,[],2,"<html><body><div><article class=""article"">\n ..."
56,15427211,9 things that I learnt from the most successfu...,2017-10-08 09:33:10,https://hackernoon.com/9-things-that-i-learnt-...,[],2,"<html><body><div><div class=""section-inner sec..."
64,15427161,Unreal Engine Improvements for Fortnite: Battl...,2017-10-08 09:07:56,https://www.unrealengine.com/en-US/blog/unreal...,[],1,"<html><body><div><div class=""blog-header-info""..."
70,15427124,StackOverflow Architecture (2016),2017-10-08 08:48:40,https://nickcraver.com/blog/2016/02/17/stack-o...,[Why Microsoft? Why IIS? Why asp.Net?<p>Was is...,48,"<html><body><div><article class=""post-content""..."


In [263]:
data.to_csv('hn_newest_youtube.csv')

In [2]:
#data = pd.read_csv('hn_newest_youtube.csv', encoding='cp1251')

## 5.2. Определение источников

Для того, чтобы определить, из каких источников собран корпус, использовался модуль urlparse (https://docs.python.org/3/library/urllib.parse.html), позволяющий выделить из URL доменное имя.

In [12]:
data_urls = []
for u in data['url']:
    data_urls.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(u)))

In [15]:
for u in sorted(list(set(data_urls))):
    print(u)

http://antrikshy.com/
http://blog.cleancoder.com/
http://evonomics.com/
http://guardian.ng/
http://highexistence.com/
http://jjjvirtual7.blogspot.com/
http://jollyrogertelephone.com/
http://mashable.com/
http://robert.ocallahan.org/
http://www.independent.co.uk/
http://www.lispcast.com/
http://www.macdrifter.com/
https://9to5google.com/
https://angular.jsnews.io/
https://bastianallgeier.com/
https://cryptoinsider.com/
https://en.wikipedia.org/
https://gizmodo.com/
https://hackernoon.com/
https://m.signalvnoise.com/
https://medium.com/
https://millcomputing.com/
https://motherboard.vice.com/
https://nickcraver.com/
https://techcrunch.com/
https://terminalsare.sexy/
https://www.cnbc.com/
https://www.currentaffairs.org/
https://www.debian.org/
https://www.iafrikan.com/
https://www.nature.com/
https://www.newscientist.com/
https://www.quantamagazine.org/
https://www.raspberrypi.org/
https://www.realclearpolitics.com/
https://www.theguardian.com/
https://www.unrealengine.com/
https://www.wi

## 5.3. Определение наиболее частых слов

Чтобы проверить, какие слова (за исключением стоп-слов) встречаются в полученном корпусе чаще всего, из сообщений сначала с помощью регулярных выражений были удалены все ненужные символы, полученные тексты были объединены в один большой, в котором после этого с помощью библиотеки nltk была рассчитана частота встречаемости каждого слова.

In [5]:
# function for removal unnecessary stuff from page content
def html_stripper(text):
    # deleting html-tags with regular expression
    text = re.sub('<[^<]+?>', '', str(text))
    # substituting all whitespaces, hyphens and dashes with simple whitespace
    text = re.sub('[\s-]', ' ', text)
    return text

In [6]:
# stripping data into one text
stripped_data = ' '.join([html_stripper(s) for s in data['url_summary']])

In [7]:
stripped_data[:200]

'    Google has finally achieved what science fiction novels (and various fundraiser campaigns) have been promising us for years: a universal translator which could close linguistic barriers between na'

In [9]:
# regular expression for identifying words
prog = re.compile('[A-Za-z]+')
l = [w for w in prog.findall(stripped_data.lower()) if not w in stop_words]
d = FreqDist(l)
print(d)
print(d.most_common(30))

<FreqDist with 9350 samples and 43721 outcomes>
[('people', 237), ('like', 230), ('one', 194), ('would', 175), ('time', 159), ('think', 151), ('says', 149), ('google', 146), ('new', 141), ('make', 129), ('also', 126), ('world', 119), ('get', 114), ('star', 114), ('even', 111), ('may', 111), ('way', 108), ('use', 106), ('could', 104), ('know', 101), ('many', 100), ('us', 97), ('work', 97), ('data', 91), ('first', 91), ('things', 91), ('years', 88), ('see', 88), ('every', 86), ('want', 82)]


## 5.4. Классификация источников по новостным и не новостным

Завершающим этапом анализа и самого проекта в целом была классификация источников по блогам и новостным изданиям и по всему остальному.

- http://antrikshy.com/
Не новости
- http://blog.cleancoder.com/ 
Блог
- http://evonomics.com/ 
Новости 
- http://guardian.ng/ 
Новости 
- http://highexistence.com/ 
Блог 
- http://jjjvirtual7.blogspot.com/ 
Блог 
- http://jollyrogertelephone.com/ 
Не новости 
- http://mashable.com/ 
Новости 
- http://robert.ocallahan.org/ 
Блог 
- http://www.independent.co.uk/ 
Новости 
- http://www.lispcast.com/ 
Блог 
- http://www.macdrifter.com/ 
Блог 
- https://9to5google.com/ 
Новости 
- https://angular.jsnews.io/ 
Новости 
- https://bastianallgeier.com/ 
Блог
- https://cryptoinsider.com/ 
Новости 
- https://en.wikipedia.org/ 
Не новости 
- https://gizmodo.com/ 
Блог 
- https://hackernoon.com/ 
Новости 
- https://m.signalvnoise.com/ 
Блог 
- https://medium.com/ 
Новости 
- https://millcomputing.com/ 
Новости 
- https://motherboard.vice.com/ 
Новости 
- https://nickcraver.com/ 
Блог 
- https://techcrunch.com/ 
Новости 
- https://terminalsare.sexy/ 
Не новости 
- https://www.cnbc.com/ 
Новости 
- https://www.currentaffairs.org/ 
Новости 
- https://www.debian.org/ 
Не новости 
- https://www.iafrikan.com/ 
Новости 
- https://www.nature.com/ 
Новости 
- https://www.newscientist.com/ 
Новости 
- https://www.quantamagazine.org/ 
Блог 
- https://www.raspberrypi.org/ 
Блог 
- https://www.realclearpolitics.com/ 
Новости 
- https://www.theguardian.com/ 
Новости 
- https://www.unrealengine.com/ 
Не новости 
- https://www.wired.com/ 
Новости 
- https://www.youtube.com/ 
Не новости

Итак, из 39 выявленных источников 12 являются блогами, 20 — новостные издания, а оставшиеся 7 ничем из упомянутого не являются. Добавим полученную информацию к полученному датасету (ссылкам на блоги и новостные издания поставим в соответствие значение '1', остальным - '0').

In [5]:
not_news = ['http://antrikshy.com/', 'http://jollyrogertelephone.com/', 'https://en.wikipedia.org/',
            'https://terminalsare.sexy/', 'https://www.debian.org/', 'https://www.unrealengine.com/', 'https://www.youtube.com/']
data['url_news'] = data['url'].apply(lambda u:
                                     1 if '{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(u)) not in not_news else 0)

In [6]:
data

,Unnamed: 0,id,title,date,url,comments,likes,url_summary,url_news
0,4,15427601,New Earbuds by Google Translates 40 Languages ...,2017-10-08 12:46:09,http://guardian.ng/life/whatsnew/new-earbuds-b...,['Things are getting closer to black mirror.'],4,"<html><body><div><article data-post-type=""post...",1
1,7,15427592,Believer's Voice of Victory Network Live Strea...,2017-10-08 12:42:57,http://jjjvirtual7.blogspot.com/2017/10/believ...,[],1,"<html><body><div><div class=""post-body entry-c...",1
2,8,15427584,Inside the internet rehab,2017-10-08 12:38:55,https://www.theguardian.com/technology/2017/ju...,[],1,"<html><body><div><div class=""content__article-...",1
3,9,15427581,New digital music platform for Afrika,2017-10-08 12:35:59,https://www.iafrikan.com/2017/10/08/okayafrica...,[],1,"<html><body><div><div class=""post-content inne...",1
4,14,15427526,How to fail in a coding interview which you ca...,2017-10-08 12:08:57,https://medium.com/@l1feh4ck/how-to-fail-in-a-...,[],2,"<html><body><div><div class=""section-inner sec...",1
5,16,15427497,Advice for New and Junior Data Scientists,2017-10-08 11:54:52,https://medium.com/@rchang/advice-for-new-and-...,[],1,"<html><body><div><div class=""section-inner sec...",1
6,45,15427322,Should Cognitect Do More for Clojure?,2017-10-08 10:25:44,http://www.lispcast.com/cognitect-clojure,[],2,"<html><body><div><article class=""article"">\r\n...",1
7,56,15427211,9 things that I learnt from the most successfu...,2017-10-08 09:33:10,https://hackernoon.com/9-things-that-i-learnt-...,[],2,"<html><body><div><div class=""section-inner sec...",1
8,64,15427161,Unreal Engine Improvements for Fortnite: Battl...,2017-10-08 09:07:56,https://www.unrealengine.com/en-US/blog/unreal...,[],1,"<html><body><div><div class=""blog-header-info""...",0
9,70,15427124,StackOverflow Architecture (2016),2017-10-08 08:48:40,https://nickcraver.com/blog/2016/02/17/stack-o...,['Why Microsoft? Why IIS? Why asp.Net?<p>Was i...,48,"<html><body><div><article class=""post-content""...",1


In [7]:
data['url_news'].value_counts()

1    39
0     7
Name: url_news, dtype: int64

Итак, из 46 собранных ссылок 39 ведут на блоги и новостные издания, а остальные — на неновостные издания.

In [8]:
data.to_csv('hn_newest_youtube.csv')

# 6. Заключение

В рамках данного проекта с сайта https://news.ycombinator.com/ были собраны последние посты при использовании официального API, написан краулер для обхода внешних ссылок, полученные страницы сохранены как текстовые сообщения, а затем отфильтрованы по упоминанию выбранного бренда. После этого был произведен краткий анализ собранного корпуса.